# 2. K-최근접이웃법(KNN)

## 2.1 핵심 개념

최근접이웃법은 각 데이터들 간의 거리를 측정하여 가장 가까운 K 개의 이웃 데이터를 참조하여 분류하는 기법이다. <u>거리는 주로 유클리디안 거리 계산법 또는 민코브스키 계산법을 사용</u>한다. 이때 **중요한 하이퍼 파라미터는 '몇 개의 케이스들을 기준으로 동일 범주로 분류 할 것인가?' 라는 K 값** 입니다.


**최적의 K 수는 일반적으로 3\~10 사이**에서 찾습니다. <u>최적의 데이터수를 찾는 계산 법은 : 데이터의 수의 제곱근 값으로 가령 1000개의 자료의 경우 sqrt(1000)=31.6 개가 최적의 K수로 제안</u> 하기도 합니다.
하지만 기준 변수의 수와 데이터 수에 따라 최적 값이라는건 달라지기 때문에 다양한 K 값을 탐색해야 합니다.


K값은 작을수록 정교한 분류와 예측이 가능하다. 하지만 **K를 1에 가깝에 조정 할 경우 훈련 데이터에서는 높은 예측율을 보일순 있으나 테스트 데이터에는 잘 맞지 않는 과대적합(overfitting)문제가 발생**할 수 있다. 반면 **K값이 크면 군집을 이루는 기준이 느슨하여 성능지표가 떨어지는 과소적합(underfitting)이 발생** 될 수 있습니다.

## 2.2 scikit-learn

KNN 은 sklearn.neighbors 패키지에 속해 있습니다.


|sklearn.neighbors|Nearest Neighbors|
|:--|:--|
|neighbors.BallTree() |BallTree for fast generalized N-point problems |
|neighbors.Distance() |DistanceMetric class|
|neighbors.KDTree() |KDTree Density Esimation|
|neighbors.KernelDestiny() |Classifier implementing the k-nearest neighbots vote.|
|neighbors.KNeighborsClassifier() |Regression based on k-nearest neighbors.|
|neighbors.KNeighborsRegressor() |Transform based on k-nearest neighbors.|
|neighbors.KNeighborsTransformer() |Transform X into a (weighted) graph of k nearest neighbors|
|neighbors.LocalOutlierFactor() |Unsupervised Outlier Detection using Local Outlier Factor(LOF)|
|neighbors.RadiusNeighborsClassifier() |Classifier implementing a vote among neighbors within a given radius|
|neighbors.RadiusNeighborsRegressor() |Regression based on neighbors within a fixed radius. |
|neighbors.RadiusNeighborsTransfomer() |Transform X into a (weighted) graph of neighbors neaerer than a radius|
|neighbors.NearestCentroid() |Nearest centroid classifier.|
|neighbors.NearestNeighbors() |Unsupervised leaner for implementing neighbor searches.|
|neighbors.NeighborhoosComponentsAnalysis() |Neighborhood Componenets Analysis|

KNeighborsClassifier 와 KNeighborsRegressor 에 대한 하위 옵션들 중에 **n_neighbors 가 K 값에 해당 하는 옵션** 입니다. **미지정시 기본 값은 5**이며, **거리측정 방식은 metric 이라는 옵션을 통해 지정** 하는데 **미지정시 기본값으로 민스코프스키(minkowski) 방식**을 따릅니다.

|Hyper Parameter||
|:--|:--|
|n_neighbors|군집으로 분류하는 구성원의 수. 기본값은 5이다.<br/>K값은 1에 가까울수록 과대적합(Overfitting) 가능성이 커지고, 크면 클수록 과소적합(Underfitting)의 가능성이 커진다.|

## 2.3 분석코드

### Part 1. 분류
데이터 로드에서 부터 독립/종속 분리, 훈련/테스트 데이터 분리작업은 동일하므로 생략합니다.

In [1]:
# 경고레벨조정
import warnings
warnings.filterwarnings("ignore")

# 데이터 로드
import pandas as pd
data = pd.read_csv("./extrafiles/breast-cancer-wisconsin.csv", encoding='utf-8')

# 컬럼정보 확인
print(data.columns)

# 독립변수/ 종속변수 분리
X = data[['Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']]
y = data[['Class']]

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['code', 'Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'],
      dtype='object')
Class    0.349609
dtype: float64
Class    0.350877
dtype: float64


In [2]:
# KNN 알고리즘 적용
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.984375

훈련 정확도가 98%로 앞서 수행한 로지스틱 회귀 모델 보다 1.2% 가량 더 좋은 결과를 보여 줍니다.

In [9]:
# 훈련데이터 혼동행렬 작성
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[331   2]
 [  6 173]]


![혼동핼렬](./extrafiles/matrix.png)

In [8]:
# 훈련데이터 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       333
           1       0.99      0.97      0.98       179

    accuracy                           0.98       512
   macro avg       0.99      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512



In [11]:
# 테스트 데이터 알고리즘 적용
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9532163742690059

In [12]:
# 테스트 데이터 혼동 행렬 작성
confusion_test = confusion_matrix(y_test, pred_test)
print("훈련데이터 오차행렬 : \n", confusion_test)

훈련데이터 오차행렬 : 
 [[106   5]
 [  3  57]]


In [13]:
# 테스트데이터 분류 레포트 작성
cfreport_test = classification_report(y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.92      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



In [18]:
# 하이퍼 파라미터 튜닝 - GridSearch
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : [1, 3, 5, 7, 9, 11]} # 하이퍼 파라미터 후보들
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [19]:
# 파라미터 튜닝 결과 확인
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Score : {:.4f}".format(grid_search.best_score_))
print("Test set Score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter : {'n_neighbors': 3}
Best Score : 0.9824
Test set Score : 0.9532


**그리드서치**
- KNieghborClassifier 최적의 하이퍼 파라미터는 n_neighbors 의 값은 11이며, 예측율은 98.24%에 달한다.
- 테스트 데이터를 이용한 예측율도 95.32%에 달한다

In [27]:
# 하이퍼 파라미터 튜닝2 - Randomized Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'n_neighbors':randint(low=1, high=20)}
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_distribs, n_iter=20, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001ABA71C5A60>})

In [28]:
# 파라미터 튜닝 결과값 확인
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Score : {:.4f}".format(random_search.best_score_))
print("Test set Score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter : {'n_neighbors': 3}
Best Score : 0.9824
Test set Score : 0.9532


**랜덤서치**
- 1에서 20까지의 구간에서 랜덤으로 선택된 하이퍼 파라미터중 가장 좋은 예측율을 보이는 값은 3이다.
- 테스트 데이터에서도 95.32%의 예측율을 보인다.


**[종합정리]**  

KNN은 심플하면서도 좋은 직관적인 방법중에 하나입니다. **K 값을 탐색하는 과정도 다른 알고리즘에 이해하기 쉽다는 장점**이 있습니다. <u>다만 KNN은 알고리즘 기반이 아닌 케이스 기반 분류 엔진입니다. 즉 입력되는 데이터에 따라 결과와 성능이 크게 달라지는 특징이 있습니다. 또한 새로운 데이터를 학습 하는 시간이 오래 걸리기 때문에 실시간 머신러닝이 필요한 분야에서는 적합하지 않을 수도 있습니다.</u> **하지만 하나의 기준 모델로 다른 알고리즘 결과와 비교하는 대상으로 사용하는데에는 예측율도 높고 사람이 이해하기 쉽기 때문에 매우 좋다**고 볼 수 있습니다.

### Part2 . 회귀(Regression)

**KNN은 분류기Classifier) 뿐 아니가 회귀자(Regressor) 알고리즘으로도 사용할 수 있습니다.** 주택가격 자료를 기준으로 회귀문제를 푸는 과정을 진행하여 보겠습니다.

In [40]:
# 데이터 로드
data2 = pd.read_csv('./extrafiles/house_price.csv', encoding='utf-8')

print(data2.columns)

X = data2[data2.columns[1:5]]
y = data2[['house_value']]

print(X.columns)

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['housing_age', 'income', 'bedrooms', 'households', 'rooms',
       'house_value'],
      dtype='object')
Index(['income', 'bedrooms', 'households', 'rooms'], dtype='object')
house_value    189260.967812
dtype: float64
house_value    188391.001357
dtype: float64


In [37]:
# KNN 알고리즘 적용
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.6804607237174459

In [38]:
# 테스트 데이터 적용
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5541889571372401

**회귀 모델에서의 설명력은 결정계수(R-Square)**이다. <u>0과 1사이의 값을 가지며, 높을수록 실제 y 와 예측값이 일치하는 좋은 모델</u>이라 할 수 있다.  
K가 기본값인 5일때, 훈련데이터는 72% 테스트 데이터는 59%의 정확도를 보여주고 있습니다.

또 **회귀 모델을 설명하는 평가지표로 RMSE** 가 있습니다. (Root Mean Squared Error) 의 약자로 오차는 적을수록 좋은 모델이라 볼 수 있습니다.

In [39]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈  련데이터 RMSE : ", np.sqrt(MSE_train))
print("테스트데이터 RMSE : ", np.sqrt(MSE_test))

훈  련데이터 RMSE :  53952.69804097723
테스트데이터 RMSE :  63831.91662964773


In [43]:
# 하이퍼 파라미터 튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : [1, 3, 5, 7, 9, 11]} # 하이퍼 파라미터 후보들
grid_search = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [44]:
# 파라미터 튜닝 결과 확인
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Score : {:.4f}".format(grid_search.best_score_))
print("Test set Score : {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best Parameter : {'n_neighbors': 11}
Best Score : 0.5638
Test set Score : 0.5880


**그리드서치**
- 그리드 서치 하이퍼파라미터 튜닝 결과 n_neighbors 값은 11이 최적 값이며, 56.38% 예측율을 기록했다.
- 테스트 데이터를 이용한 예측율은 58.80%에 달하였다.

In [47]:
# 하이퍼 파라미터 튜닝2 - Randomized Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'n_neighbors':randint(low=1, high=20)}
random_search = RandomizedSearchCV(KNeighborsRegressor(), param_distributions=param_distribs, n_iter=20, cv=5) #n_iter : 랜덤서치 탐색 횟수
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001ABA71E61C0>})

In [48]:
# 파라미터 튜닝 결과값 확인
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Score : {:.4f}".format(random_search.best_score_))
print("Test set Score : {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter : {'n_neighbors': 19}
Best Score : 0.5777
Test set Score : 0.6004


**랜덤 서치**
- 1에서 20까지의 구간에서 랜덤으로 선택된 하이퍼 파라미터중 가장 좋은 예측율을 보이는 값은 19이다.
- 테스트 데이터에서도 60.04%의 예측율을 보인다.

**[종합정리]**  
회귀에 사용되는 주택가격 데이터는 16,000여개로 데이터 수가 많습니다. 이 경우 **K값을 더 늘려 가까운 주변 여러 데이터의 평균으로 주택 가격을 예측하는 것이 모델의 성능 및 일반화에 도움이 되는것으로 파악**되었습니다. 1~20의 범위를 조금 더 늘려서 그리드 탐색이나 랜덤 탐색을 시도해도 무방합니다. 즉 KNN은 데이터의 수, 특성치의 수에 따라 K 값을 작게 또는 크게 늘려야 하는 인스턴드 기반 모델입니다.